
# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Business Problem section

### Background

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.


### Research Question

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.



## Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

# Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling



### 1. Collect Inspection Data

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:


In [60]:
!pip install folium
!pip install geopy

import numpy as np
import pandas as pd
import datetime as dt 
import json 
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 


from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 

print('Libraries imported.')

Libraries imported.


In [61]:
df_HousePrice = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:


In [62]:
df_HousePrice.head()

,{79A74E22-41E2-1289-E053-6B04A8C01627},60000,2018-06-29 00:00,DH3 1DN,F,N,L,20,Unnamed: 8,BEACONSFIELD TERRACE,BIRTLEY,CHESTER LE STREET,GATESHEAD,TYNE AND WEAR,B,A
0,{79A74E22-41E3-1289-E053-6B04A8C01627},149950,2018-06-14 00:00,DH4 6NZ,T,Y,F,50,NaN,GLANVILLE DRIVE,NaN,HOUGHTON LE SPRING,SUNDERLAND,TYNE AND WEAR,A,A
1,{79A74E22-41E4-1289-E053-6B04A8C01627},164950,2018-06-29 00:00,SR2 0FD,S,Y,F,6,NaN,WILSHIRE CLOSE,NaN,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{79A74E22-41E5-1289-E053-6B04A8C01627},224950,2018-06-29 00:00,SR2 0FA,D,Y,F,47,NaN,WOODHAM DRIVE,NaN,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
3,{79A74E22-41E6-1289-E053-6B04A8C01627},129950,2018-06-28 00:00,DH4 6NY,S,Y,F,65A,NaN,CHALK HILL ROAD,NaN,HOUGHTON LE SPRING,SUNDERLAND,TYNE AND WEAR,A,A
4,{79A74E22-41E7-1289-E053-6B04A8C01627},144395,2018-02-23 00:00,NE31 2EL,T,Y,F,9,NaN,TURNBERRY DRIVE,NaN,HEBBURN,SOUTH TYNESIDE,TYNE AND WEAR,A,A


In [63]:
df_HousePrice.shape

(1031509, 16)

Our dataset consists of over 1031509 rows and 16 columns. We will now prepare and preprocess data accordingly.

### 3. Data preparation and preprocessing

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

   1. Rename the column names
   2. Format the date column
   3. Sort data by date of sale
   4. Select data only for the city of London
   5. Make a list of street names in London
   6. Calculate the street-wise average price of the property
   7. Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
   8. Join the data to find the coordinates of locations which fit into client's budget
   9. Plot recommended locations on London map along with current market prices


In [64]:
df_HousePrice.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [65]:
df_HousePrice['Date_Transfer'] = df_HousePrice['Date_Transfer'].apply(pd.to_datetime)

df_HousePrice.drop(df_HousePrice[df_HousePrice.Date_Transfer.dt.year < 2018].index, inplace=True)

df_HousePrice.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

df_HousePrice_london = df_HousePrice.query("Town_City == 'LONDON'")

streets = df_HousePrice_london['Street'].unique().tolist()

In [66]:
df_HousePrice_price = df_HousePrice_london.groupby(['Street'])['Price'].mean().reset_index()

df_HousePrice_price.columns = ['Street', 'Avg_Price']


df_HousePrice_affordable = df_HousePrice_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")


df_HousePrice_affordable


,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
857,BARONSMEAD ROAD,2375000.0
...,...,...
13733,WILFRED STREET,2410538.5
13759,WILLOW BRIDGE ROAD,2425000.0
13779,WILSON STREET,2257500.0
13808,WINCHENDON ROAD,2350000.0


In [67]:
import hmac

from geopy.distance import geodesic
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

In [68]:
for index, item in df_HousePrice_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 857
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 857, dtype: object
item.Street only: BARONSMEAD ROAD
index: 983
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 983, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1105
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1105, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1218
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [72]:
geolocator = Nominatim(user_agent="Hemendra_jasani")

In [71]:
df_HousePrice_affordable['city_coord'] = df_HousePrice_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [73]:
df_HousePrice_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(29.7127696, -98.0948057)"
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2400000.0,"(53.749244, -0.32678)"
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"
...,...,...,...
13733,WILFRED STREET,2410538.5,"(42.5083498, -82.9217946)"
13759,WILLOW BRIDGE ROAD,2425000.0,"(53.6408743, -1.2004617)"
13779,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)"
13808,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)"


In [74]:
df_HousePrice_affordable[['Latitude', 'Longitude']] = df_HousePrice_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [75]:
df_HousePrice_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(29.7127696, -98.0948057)",29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,"(53.749244, -0.32678)",53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457
...,...,...,...,...,...
13733,WILFRED STREET,2410538.5,"(42.5083498, -82.9217946)",42.508350,-82.921795
13759,WILLOW BRIDGE ROAD,2425000.0,"(53.6408743, -1.2004617)",53.640874,-1.200462
13779,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)",45.700486,-121.521875
13808,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)",42.645331,-71.954714


In [76]:
df_HousePrice_affordable1 = df_HousePrice_affordable.drop(columns=['city_coord'])
df_HousePrice_affordable1

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457
...,...,...,...,...
13733,WILFRED STREET,2410538.5,42.508350,-82.921795
13759,WILLOW BRIDGE ROAD,2425000.0,53.640874,-1.200462
13779,WILSON STREET,2257500.0,45.700486,-121.521875
13808,WINCHENDON ROAD,2350000.0,42.645331,-71.954714


In [77]:
address = 'London, UK'

geolocator = Nominatim(user_agent="Hemendra_jasani")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [78]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, price, street in zip(df_HousePrice_affordable1['Latitude'], df_HousePrice_affordable1['Longitude'], df_HousePrice_affordable1['Avg_Price'], df_HousePrice_affordable1['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [85]:
CLIENT_ID = '2SDYKK3MF51JCT0P3LMDW2QSHAG5J3ESNOIWM01JTISI0LB2' 
CLIENT_SECRET = 'RPF4SF1YYSTKKUQ4D0C3SU4TO0QTEEXVK5JCNM3X2LBWLRIQ' 
VERSION = '20201029'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2SDYKK3MF51JCT0P3LMDW2QSHAG5J3ESNOIWM01JTISI0LB2
CLIENT_SECRET:RPF4SF1YYSTKKUQ4D0C3SU4TO0QTEEXVK5JCNM3X2LBWLRIQ


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.


In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
loc_London_venues = getNearbyVenues(names=df_HousePrice_affordable1['Street'],
                                   latitudes=df_HousePrice_affordable1['Latitude'],
                                   longitudes=df_HousePrice_affordable1['Longitude']
                                  )


ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENS

In [95]:
loc_London_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
...,...,...,...,...,...,...,...
4313,WILSON STREET,45.700486,-121.521875,Shell,45.696831,-121.523032,Gas Station
4314,WILSON STREET,45.700486,-121.521875,Orchard House,45.701669,-121.526546,Food
4315,WINGATE ROAD,41.043293,-80.621035,Belleria Pizza,41.044029,-80.615388,Italian Restaurant
4316,WINGATE ROAD,41.043293,-80.621035,Ohio Slopes,41.041855,-80.622971,Ski Area


In [96]:
loc_London_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,28,28,28,28,28,28
ANHALT ROAD,5,5,5,5,5,5
ANSDELL TERRACE,50,50,50,50,50,50
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,15,15,15,15,15,15
...,...,...,...,...,...,...
WESTMORELAND PLACE,16,16,16,16,16,16
WHITFIELD STREET,16,16,16,16,16,16
WILFRED STREET,13,13,13,13,13,13


In [97]:
print('There are {} uniques categories.'.format(len(loc_London_venues['Venue Category'].unique())))

There are 344 uniques categories.


In [98]:
loc_London_venues.shape

(4318, 7)

In [99]:
London_venues_hotspot = pd.get_dummies(loc_London_venues[['Venue Category']], prefix="", prefix_sep="")


London_venues_hotspot['Street'] = location_venues['Street'] 


fixed_columns = [London_venues_hotspot.columns[-1]] + list(London_venues_hotspot.columns[:-1])


London_venues_hotspot = London_venues_hotspot[fixed_columns]

London_venues_hotspot.head()

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
london_venues_group = London_venues_hotspot.groupby('Street').mean().reset_index()
london_venues_group

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ANHALT ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ANSDELL TERRACE,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,APPLEGARTH ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BARONSMEAD ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,WESTMORELAND PLACE,0.000000,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,...,0.0,0.062500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,WHITFIELD STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,WILFRED STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,WILSON STREET,0.055556,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
num_top_venues = 5

for hood in london_venues_group['Street']:
    print("----"+hood+"----")
    temp = london_venues_group[london_venues_group['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1  Indian Restaurant  0.07
2                Pub  0.07
3                Bar  0.07
4        Coffee Shop  0.07


----ANHALT ROAD----
           venue  freq
0  Movie Theater   0.2
1          Hotel   0.2
2   Intersection   0.2
3            Gym   0.2
4    Coffee Shop   0.2


----ANSDELL TERRACE----
                venue  freq
0           Juice Bar  0.08
1          Restaurant  0.06
2      Clothing Store  0.06
3  Italian Restaurant  0.06
4               Hotel  0.06


----APPLEGARTH ROAD----
             venue  freq
0           Casino   0.2
1        Nightclub   0.2
2  Auto Dealership   0.2
3   Sandwich Place   0.2
4              Bar   0.2


----BARONSMEAD ROAD----
                 venue  freq
0    Food & Drink Shop  0.13
1       Breakfast Spot  0.07
2      Thai Restaurant  0.07
3  Indie Movie Theater  0.07
4     Community Center  0.07


----BEAUCLERC ROAD----
             venue  freq
0      Pizza Place   0.4
1          

In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [130]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_venues_group['Street']

for ind in np.arange(london_venues_group.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_venues_group.iloc[ind, :], num_top_venues)

In [131]:
venues_sorted.shape

(142, 11)

In [132]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Restaurant,Coffee Shop,Pub,Bar,Park,Burger Joint,Museum,Art Gallery
1,ANHALT ROAD,Hotel,Movie Theater,Gym,Intersection,Coffee Shop,Factory,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
2,ANSDELL TERRACE,Juice Bar,Restaurant,Hotel,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Café,Sporting Goods Shop,French Restaurant
3,APPLEGARTH ROAD,Bar,Sandwich Place,Auto Dealership,Nightclub,Casino,Fabric Shop,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
4,BARONSMEAD ROAD,Food & Drink Shop,Pizza Place,Indie Movie Theater,Restaurant,Nature Preserve,Breakfast Spot,Coffee Shop,Sports Club,Thai Restaurant,Farmers Market


In [141]:
london_venues_group=df_HousePrice_affordable1

In [142]:
kclusters = 5

london_grouped_clustering = london_venues_group.drop('Street', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)


kmeans.labels_[0:50]

array([2, 0, 1, 0, 3, 2, 3, 1, 1, 2, 0, 0, 2, 3, 3, 2, 0, 1, 2, 4, 4, 0,
       2, 2, 1, 0, 4, 2, 1, 0, 3, 0, 3, 3, 4, 0, 0, 3, 1, 2, 3, 4, 1, 4,
       1, 1, 4, 1, 1, 0], dtype=int32)

In [143]:
london_grouped_clustering=df_HousePrice_affordable1
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [144]:
london_grouped_clustering.shape

(159, 4)

In [145]:
df_HousePrice_affordable1.shape

(159, 4)

In [146]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [147]:
df_HousePrice_affordable1.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [148]:
london_grouped_clustering['Cluster Labels'] = kmeans.labels_


london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) 

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,2,Café,Indian Restaurant,Restaurant,Coffee Shop,Pub,Bar,Park,Burger Joint,Museum,Art Gallery
390,ANHALT ROAD,2.435000e+06,29.712770,-98.094806,0,Hotel,Movie Theater,Gym,Intersection,Coffee Shop,Factory,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,1,Juice Bar,Restaurant,Hotel,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Café,Sporting Goods Shop,French Restaurant
422,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,0,Bar,Sandwich Place,Auto Dealership,Nightclub,Casino,Fabric Shop,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
857,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Food & Drink Shop,Pizza Place,Indie Movie Theater,Restaurant,Nature Preserve,Breakfast Spot,Coffee Shop,Sports Club,Thai Restaurant,Farmers Market
983,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,2,Pizza Place,Spa,Automotive Shop,Harbor / Marina,Factory,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
1105,BELVEDERE DRIVE,2.340000e+06,38.072439,-78.459970,3,Pool,Playground,Athletics & Sports,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
1218,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,1,Coffee Shop,Italian Restaurant,Hotel,Café,Gastropub,Pizza Place,Chinese Restaurant,Bar,Bakery,Pub
1256,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,1,Pub,Lake,Coffee Shop,French Restaurant,Bakery,Chinese Restaurant,Train Station,Brewery,Pizza Place,Cricket Ground
1556,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,2,Home Service,Zoo,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit


In [149]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [150]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Hotel,Movie Theater,Gym,Intersection,Coffee Shop,Factory,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
422,2400000.0,Bar,Sandwich Place,Auto Dealership,Nightclub,Casino,Fabric Shop,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant
1635,2397132.0,Convenience Store,Coffee Shop,Breakfast Spot,Grocery Store,Park,Coworking Space,Falafel Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
1800,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,2425000.0,Clothing Store,Electronics Store,Chinese Restaurant,Gym,Department Store,Basketball Court,Stationery Store,American Restaurant,Convenience Store,Women's Store


In [151]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Juice Bar,Restaurant,Hotel,Clothing Store,Italian Restaurant,Pub,Indian Restaurant,Café,Sporting Goods Shop,French Restaurant
1218,2208500.0,Coffee Shop,Italian Restaurant,Hotel,Café,Gastropub,Pizza Place,Chinese Restaurant,Bar,Bakery,Pub
1256,2217000.0,Pub,Lake,Coffee Shop,French Restaurant,Bakery,Chinese Restaurant,Train Station,Brewery,Pizza Place,Cricket Ground
2229,2200000.0,Trail,Zoo,Dry Cleaner,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
2642,2250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Restaurant,Coffee Shop,Pub,Bar,Park,Burger Joint,Museum,Art Gallery
983,2480000.0,Pizza Place,Spa,Automotive Shop,Harbor / Marina,Factory,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
1556,2456875.0,Home Service,Zoo,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
1983,2492500.0,Supermarket,English Restaurant,Park,Gym,Dry Cleaner,Hardware Store,Rental Car Location,Coffee Shop,Discount Store,American Restaurant
2139,2461000.0,Pub,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [156]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
857,2375000.0,Food & Drink Shop,Pizza Place,Indie Movie Theater,Restaurant,Nature Preserve,Breakfast Spot,Coffee Shop,Sports Club,Thai Restaurant,Farmers Market
1105,2340000.0,Pool,Playground,Athletics & Sports,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
2071,2375000.0,Pub,Park,Indian Restaurant,Pizza Place,Hotel,Yoga Studio,Breakfast Spot,Juice Bar,Outdoor Sculpture,Sushi Restaurant
2132,2379652.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2948,2367500.0,Hotel,Pub,Garden,Café,Coffee Shop,Chinese Restaurant,Bar,Cocktail Bar,Italian Restaurant,Mediterranean Restaurant


In [157]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2246,2.300000e+06,Farm,Falafel Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory
2410,2.286679e+06,Café,Pub,Italian Restaurant,Bar,Coffee Shop,Convenience Store,Park,French Restaurant,Supermarket,Portuguese Restaurant
2690,2.287500e+06,Pub,Reservoir,Gift Shop,Harbor / Marina,Art Museum,Brewery,Coworking Space,Factory,Electronics Store,English Restaurant
3381,2.298000e+06,Hotel,Zoo,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
4289,2.265000e+06,Pub,Factory,Egyptian Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


# Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.


# Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.
